## TP4 Machine learning 
réalisé par : Mohamed Karim HOSNI et Slim El Arbi 

In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import urllib.request
import pandas as pd
pd.options.mode.chained_assignment = None  
from io import StringIO


In [19]:
from matplotlib import pyplot as plt
from datetime import timedelta
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [20]:
def get_melbourne_data() -> pd.DataFrame:
  
    raw_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv" 
    response = urllib.request.urlopen(raw_url)
    response = response.read().decode('utf-8')
    
    data = pd.read_csv(StringIO(response))
    data['Date'] = pd.to_datetime(data['Date'])

    return data


def split_train_test_data(melbourne_data: pd.DataFrame, split_year: str="1987") -> (pd.DataFrame, pd.DataFrame):

    split_year = "{}".format(int(split_year) - 1)

    train_data = melbourne_data.loc[:split_year]

    test_data = melbourne_data.loc[split_year:]

    return train_data, test_data

def build_training_point(data, t_str, history_days=64, horizon_days=1):
    
    t_datetime = datetime.strptime(t_str, "%Y-%m-%d 00:00:00")

    try:
        x = data.loc[t_datetime - timedelta(days=history_days - 1):t_datetime]
        y = data.loc[t_datetime + timedelta(days=1):t_datetime + timedelta(days=horizon_days)]
    except KeyError:
        raise KeyError("The date {} is not in the data".format(t_str))

    # Return
    return x, y


def create_training_points(data, history_days=64, horizon_days=32):

    X = []
    Y = []
    for t in data.index[history_days:(len(data) - horizon_days)]:
        try:
            x, y = build_training_point(data, str(t), history_days=history_days, horizon_days=horizon_days)
            if (len(x) == history_days) & (len(y) == horizon_days):
                X.append(x)
                Y.append(y)
        except KeyError:
            continue
    X = np.stack(X)
    Y = np.stack(Y)
    return X, Y

melbourne_data = get_melbourne_data()

In [21]:
print(melbourne_data.head())

        Date  Temp
0 1981-01-01  20.7
1 1981-01-02  17.9
2 1981-01-03  18.8
3 1981-01-04  14.6
4 1981-01-05  15.8


In [22]:
def split_data(year):
  split_date = pd.datetime(year,1,1)
  train_data = melbourne_data.loc[melbourne_data['Date'] < split_date]
  test_data = melbourne_data.loc[melbourne_data['Date'] >= split_date]
  scaler = MinMaxScaler()
  train_data[["Temp"]] = scaler.fit_transform(train_data[["Temp"]] )
  test_data[["Temp"]] = scaler.fit_transform(test_data[["Temp"]] )
  train_data.set_index('Date', inplace=True)
  test_data.set_index('Date', inplace=True)
  return train_data, test_data

In [23]:
from keras.layers import Input, LSTM, Dense, Flatten
from keras.models import Model

def create_model(history_days, horizon_days):
  model_input = Input(shape=(history_days, 1), name='x', dtype='float32')
  z = LSTM(64, activation='relu', return_sequences=True)(model_input)
  z = Flatten()(z) 
  z = Dense(horizon_days, activation='linear')(z)

  # Keras model
  model_keras = Model(inputs=model_input, outputs=z)

  return model_keras

model_3m = create_model(90, 30)
model_6m = create_model(180, 30)
model_12m = create_model(365, 30)

In [24]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=90, horizon_days=30)
model_3m.compile(optimizer='adam', loss='mse')
model_3m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
61/61 [==============================] - 4s 41ms/step - loss: 0.0294
Epoch 2/32
61/61 [==============================] - 3s 42ms/step - loss: 0.0139
Epoch 3/32
61/61 [==============================] - 2s 41ms/step - loss: 0.0130
Epoch 4/32
61/61 [==============================] - 3s 42ms/step - loss: 0.0128
Epoch 5/32
61/61 [==============================] - 3s 51ms/step - loss: 0.0124
Epoch 6/32
61/61 [==============================] - 3s 41ms/step - loss: 0.0123
Epoch 7/32
61/61 [==============================] - 3s 41ms/step - loss: 0.0122
Epoch 8/32
61/61 [==============================] - 3s 42ms/step - loss: 0.0122
Epoch 9/32
61/61 [==============================] - 3s 41ms/step - loss: 0.0120
Epoch 10/32
61/61 [==============================] - 3s 41ms/step - loss: 0.0120
Epoch 11/32
61/61 [==============================] - 2s 41ms/step - loss: 0.0121
Epoch 12/32
61/61 [==============================] - 3s 41ms/step - loss: 0.0119
Epoch 13/32
61/61 [==================

In [25]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1987, k=3: ", mean_squared_error(y_pred_3m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1988, k=3: ", mean_squared_error(y_pred_3m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=90, horizon_days=30)
y_pred_3m = model_3m.predict(X_test)
print("1989, k=3: ", mean_squared_error(y_pred_3m[0], Y_train[2]))

1987, k=3:  0.027015956995034316
1988, k=3:  0.011407004267776893
1989, k=3:  0.026176998032545997


In [26]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=180, horizon_days=30)
model_6m.compile(optimizer='adam', loss='mse')
model_6m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
56/56 [==============================] - 6s 81ms/step - loss: 0.0302
Epoch 2/32
56/56 [==============================] - 4s 80ms/step - loss: 0.0121
Epoch 3/32
56/56 [==============================] - 4s 79ms/step - loss: 0.0121
Epoch 4/32
56/56 [==============================] - 5s 81ms/step - loss: 0.0116
Epoch 5/32
56/56 [==============================] - 4s 80ms/step - loss: 0.0113
Epoch 6/32
56/56 [==============================] - 5s 82ms/step - loss: 0.0112
Epoch 7/32
56/56 [==============================] - 5s 81ms/step - loss: 0.0111
Epoch 8/32
56/56 [==============================] - 5s 88ms/step - loss: 0.0109
Epoch 9/32
56/56 [==============================] - 4s 80ms/step - loss: 0.0107
Epoch 10/32
56/56 [==============================] - 5s 81ms/step - loss: 0.0106
Epoch 11/32
56/56 [==============================] - 4s 80ms/step - loss: 0.0106
Epoch 12/32
56/56 [==============================] - 4s 80ms/step - loss: 0.0105
Epoch 13/32
56/56 [==================

In [27]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1987, k=6: ", mean_squared_error(y_pred_6m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1988, k=6: ", mean_squared_error(y_pred_6m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=180, horizon_days=30)
y_pred_6m = model_6m.predict(X_test)
print("1989, k=6: ", mean_squared_error(y_pred_6m[0], Y_train[2]))

1987, k=6:  0.008685744760261092
1988, k=6:  0.009408929715739653
1989, k=6:  0.011483166456616822


In [28]:
train_data, test_data = split_data(1987)
X_train, Y_train = create_training_points(train_data, history_days=365, horizon_days=30)
model_12m.compile(optimizer='adam', loss='mse')
model_12m.fit(X_train, Y_train, epochs=32, batch_size=32, verbose=1)

Epoch 1/32
44/44 [==============================] - 8s 158ms/step - loss: 0.0258
Epoch 2/32
44/44 [==============================] - 7s 157ms/step - loss: 0.0120
Epoch 3/32
44/44 [==============================] - 7s 160ms/step - loss: 0.0115
Epoch 4/32
44/44 [==============================] - 7s 157ms/step - loss: 0.0110
Epoch 5/32
44/44 [==============================] - 7s 158ms/step - loss: 0.0107
Epoch 6/32
44/44 [==============================] - 7s 164ms/step - loss: 0.0103
Epoch 7/32
44/44 [==============================] - 7s 160ms/step - loss: 0.0100
Epoch 8/32
44/44 [==============================] - 7s 159ms/step - loss: 0.0099
Epoch 9/32
44/44 [==============================] - 7s 161ms/step - loss: 0.0096
Epoch 10/32
44/44 [==============================] - 7s 160ms/step - loss: 0.0094
Epoch 11/32
44/44 [==============================] - 7s 161ms/step - loss: 0.0091
Epoch 12/32
44/44 [==============================] - 7s 159ms/step - loss: 0.0089
Epoch 13/32
44/44 [======

In [29]:
train_data, test_data = split_data(1987)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1987, k=12: ", mean_squared_error(y_pred_12m[0], Y_test[0]))

train_data, test_data = split_data(1988)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1988, k=12: ", mean_squared_error(y_pred_12m[0], Y_train[1]))

train_data, test_data = split_data(1989)
X_test, Y_test = create_training_points(test_data, history_days=365, horizon_days=30)
y_pred_12m = model_12m.predict(X_test)
print("1989, k=12: ", mean_squared_error(y_pred_12m[0], Y_train[2]))

1987, k=12:  0.038549122123843024
1988, k=12:  0.023159186670343206
1989, k=12:  0.053148860747176015


1987, k=12:  0.03865884223545268
1988, k=12:  0.01985055503939519
1989, k=12:  0.04430945823737449

Metrics: Mean Squared Error

1987, k=3: 0.027015956995034316 1988, k=3: 0.011407004267776893 1989, k=3: 0.026176998032545997

1987, k=6: 0.008685744760261092 1988, k=6: 0.009408929715739653 1989, k=6: 0.011483166456616822

1987, k=12: 0.038549122123843024 1988, k=12: 0.023159186670343206 1989, k=12: 0.053148860747176015


Le model le plus précis est le model est k=6.
